# Generate graphRAG for Financial Data #

## Imports ##

In [1]:
import os
import glob
import time
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path
from typing import List

from langchain_community.document_loaders import TextLoader
from langchain_community.graphs import Neo4jGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI, OpenAIEmbeddings, AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_text_splitters import TokenTextSplitter
from neo4j.exceptions import ClientError

# import openai
# from openai import OpenAI
# from openai import AzureOpenAI

from time import sleep
import hashlib



## Initializations ##

In [10]:
# Load from environment
load_dotenv('.env', override=True)

AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY"), 



# Embeddings & LLM models
embedding_dimension = 1536
embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-3",api_version="2024-02-01",dimensions=embedding_dimension)

llm = AzureChatOpenAI(azure_deployment='chat_gtp_35',api_version="2023-05-15", temperature=0)

# Get Neo4j credentials from environment variables
NEO4J_URI=os.getenv("NEO4J_URI")
NEO4J_USERNAME=os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD=os.getenv("NEO4J_PASSWORD")

graph = Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
sleep(5)

#Clear KG from previous sesions
graph.refresh_schema()
graph.query("MATCH (n) DETACH DELETE n")
graph.query("MATCH (n) DETACH DELETE n")
graph.query("DROP INDEX hypothetical_questions IF EXISTS")
graph.query("DROP INDEX parent_document IF EXISTS")
graph.query("DROP INDEX summary IF EXISTS")
graph.query("DROP INDEX typical_rag IF EXISTS")
# graph.query("""
#   SHOW VECTOR INDEXES
#   """
# )

class Questions(BaseModel):
    """Generating hypothetical questions about text."""

    questions: List[str] = Field(
        ...,
        description=(
            "Generated hypothetical questions based on " "the information from the text"
        ),
    )


questions_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are generating a maximum of 5 hypothetical questions based on the information and return the output in JSON format "
                "Make sure to provide full context in the generated "
                "questions but again only 5 questions are required."
            ),
        ),
        (
            "human",
            (
                "Use the given format to generate hypothetical questions from the"
                "following input: {input}"
            ),
        ),
    ]
)

question_chain = questions_prompt | llm.with_structured_output(Questions.json)

# Ingest summaries

summary_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are generating concise and accurate summaries based on the "
                "information found in the text."
            ),
        ),
        (
            "human",
            ("Generate a summary of the following input: {question}\n" "Summary:"),
        ),
    ]
)

summary_chain = summary_prompt | llm

def clean_questions_data(questions_data):
    if 'questions' not in questions_data:
        return []

    cleaned_questions = []
    for entry in questions_data['questions']:
        if isinstance(entry, dict) and 'question' in entry:
            cleaned_questions.append(entry)
        elif isinstance(entry, str):
            cleaned_questions.append({'question': entry})
        else:
            # Handle cases where the entry is not a string or doesn't contain the 'question' key
            print(f"Invalid entry found and skipped: {entry}")
    # Convert the cleaned_questions list into a pandas DataFrame
    df = pd.DataFrame(cleaned_questions)

    # Generate a unique file ID using the current timestamp
    import datetime
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    file_id = f"questions_{timestamp}"

    # Define the folder path
    folder_path = "./LLM_questions"

    # Create the folder if it doesn't exist
    os.makedirs(folder_path, exist_ok=True)
    
    # Save the DataFrame into a CSV file with the unique file ID
    df.to_csv(f"./LLM_questions/{file_id}.csv", index=False)
    return cleaned_questions

In [16]:
# Load from environment
load_dotenv('.env', override=True)

AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY"), 



# Embeddings & LLM models
embedding_dimension = 1536
embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-3",api_version="2024-02-01",dimensions=embedding_dimension)

llm = AzureChatOpenAI(azure_deployment='chat_gtp_35',api_version="2023-05-15", temperature=0)

# Get Neo4j credentials from environment variables
NEO4J_URI=os.getenv("NEO4J_URI")
NEO4J_USERNAME=os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD=os.getenv("NEO4J_PASSWORD")

graph = Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
sleep(5)

#Clear KG from previous sesions
graph.refresh_schema()
graph.query("MATCH (n) DETACH DELETE n")
graph.query("MATCH (n) DETACH DELETE n")
graph.query("DROP INDEX hypothetical_questions IF EXISTS")
graph.query("DROP INDEX parent_document IF EXISTS")
graph.query("DROP INDEX summary IF EXISTS")
graph.query("DROP INDEX typical_rag IF EXISTS")
# graph.query("""
#   SHOW VECTOR INDEXES
#   """
# )

class Questions(BaseModel):
    """Generating hypothetical questions about text."""

    questions: List[str] = Field(
        ...,
        description=(
            "Generated hypothetical questions based on " "the information from the text"
        ),
    )


questions_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are generating a maximum of 5 hypothetical questions based on the information and return the output in JSON format "
                "Make sure to provide full context in the generated "
                "questions but again only 5 questions are required."
            ),
        ),
        (
            "human",
            (
                "Use the given format to generate hypothetical questions from the"
                "following input: {input}"
            ),
        ),
    ]
)

question_chain = questions_prompt | llm.with_structured_output(Questions.json)

# Ingest summaries

summary_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are generating concise and accurate summaries based on the "
                "information found in the text."
            ),
        ),
        (
            "human",
            ("Generate a summary of the following input: {question}\n" "Summary:"),
        ),
    ]
)

summary_chain = summary_prompt | llm

def clean_questions_data(questions_data, folder_name, file_name):
    if 'questions' not in questions_data:
        return []

    cleaned_questions = []
    for entry in questions_data['questions']:
        if isinstance(entry, dict) and 'question' in entry:
            cleaned_questions.append(entry)
        elif isinstance(entry, str):
            cleaned_questions.append({'question': entry})
        else:
            # Handle cases where the entry is not a string or doesn't contain the 'question' key
            print(f"Invalid entry found and skipped: {entry}")

    # Convert the cleaned_questions list into a pandas DataFrame
    df = pd.DataFrame(cleaned_questions)

    # Add a new column with the folder name and file name
    df['source'] = f"{folder_name}/{file_name}"

    # Generate a unique file ID using the current timestamp
    import datetime
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    file_id = f"questions_{timestamp}"

    # Define the folder path
    folder_path = "./LLM_questions"

    # Create the folder if it doesn't exist
    os.makedirs(folder_path, exist_ok=True)
    
    # Save the DataFrame into a CSV file with the unique file ID
    df.to_csv(f"./LLM_questions/{file_id}.csv", index=False)
    return cleaned_questions

## Data ##

In [11]:
# import getFinancialInfo_3

# com_list = pd.read_csv('ticker_select_test.txt', sep='\t',names=['symbol', 'cik'])
# data = getFinancialInfo_3.get_data(com_list,True)
# data

In [16]:
# pip install sec-api
# from sec_api import ExtractorApi
# import multiprocessing

# API_KEY = =os.getenv("SEC_API")

# extractorApi = ExtractorApi(API_KEY)

# def extract_items_10k(filing_url):
#   items = ["1", "1A", "7",
#            "7A"]
#   filing_name = os.path.basename(filing_url)

#   for item in items:
#     print("item:", item, "url", filing_url)

#     try:
#       section_text = extractorApi.get_section(filing_url=filing_url,
#                                               section=item,
#                                               return_type="text")

#       # do something with section_text. for example, save to disk, in a database
#       # or perform analytics
#       # IMPORTANT: you don't want to hold a large number of sections in memory
#       # and add sections to a list. otherwise you end up with out-of-memory issues.
#       # instead make sure to let the garbage collection release memory frequently.

#       # Create a new directory for the current filing if it doesn't exist
#       if not os.path.exists(filing_name):
#         os.makedirs(filing_name)

#       # Save the extracted text to a file
#       with open(f"{filing_name}/{item}.txt", "w") as f:
#         f.write(section_text)
#     except Exception as e:
#       print(e)

# urls_10k = ["https://www.sec.gov/Archives/edgar/data/789019/000095017023035122/msft-20230630.htm"]

# # represents the number of processes to run in parallel.
# # if you perform a CPU-light task such as saving a section to your local disk,
# # you might want to set the number of processes slighly higher
# # than number of CPU cores. if you perform CPU-heavy tasks, set number_of_processes
# # to the actual number of CPU cores.
# number_of_processes = 2

# with multiprocessing.Pool(number_of_processes) as pool:
#   pool.map(extract_items_10k, urls_10k)

## Code ##

In [17]:
def extract_entities_relationships(folder):
    files = glob.glob(f'./sec_10K_data/{folder}/*.txt')
    start = time.perf_counter()
    print(f"Running pipeline for {len(files)} files in {folder} folder")

    # Iterate over the list of file paths and print each one
    for file_path in files:
        file_name = os.path.basename(file_path)
        print(f"Extracting entities and relationships for: {str(file_path)}")
    
        # Load the text file
        loader = TextLoader(str(file_path), encoding='utf-8')
        documents = loader.load()
       
        # Ingest Parent-Child node pairs
        parent_splitter = TokenTextSplitter(chunk_size=512*5, chunk_overlap=24)
        child_splitter = TokenTextSplitter(chunk_size=100*5, chunk_overlap=24)
        parent_documents = parent_splitter.split_documents(documents)
        
        for i, parent in enumerate(parent_documents):
            child_documents = child_splitter.split_documents([parent])
            params = {
                "parent_text": parent.page_content,
                "parent_id": f"{file_path}-{i}",  # Use file path and index to create a unique ID
                "parent_embedding": embeddings.embed_query(parent.page_content),
                "children": [
                    {
                        "text": c.page_content,
                         "id": f"{file_path}-{i}-{ic}",  # Use file path, parent index, and child index
                        "embedding": embeddings.embed_query(c.page_content),
                    }
                    for ic, c in enumerate(child_documents)
                ],
            }
            # Ingest data
            graph.query(
                """
               MERGE (p:Parent {id: $parent_id})
                ON CREATE SET p.text = $parent_text
                WITH p
                CALL db.create.setVectorProperty(p, 'embedding', $parent_embedding)
                YIELD node
                WITH p 
                UNWIND $children AS child
                MERGE (c:Child {id: child.id})
                ON CREATE SET c.text = child.text
                MERGE (c)<-[:HAS_CHILD]-(p)
                WITH c, child
                CALL db.create.setVectorProperty(c, 'embedding', child.embedding)
                YIELD node
                RETURN count(*)
                """,
                params,
            )
            # Create vector index for child
            try:
                graph.query(
                    "CALL db.index.vector.createNodeIndex('parent_document', "
                    "'Child', 'embedding', $dimension, 'cosine')",
                    {"dimension": embedding_dimension},
                )
            except ClientError:  # already exists
                pass
            # Create vector index for parents
            try:
                graph.query(
                    "CALL db.index.vector.createNodeIndex('typical_rag', "
                    "'Parent', 'embedding', $dimension, 'cosine')",
                    {"dimension": embedding_dimension},
                )
            except ClientError:  # already exists
                pass

        for i, parent in enumerate(parent_documents):
            questions_data = question_chain.invoke(parent.page_content)
            cleaned_questions_list = clean_questions_data(questions_data, folder, file_name)
            if 'questions' in questions_data:
                questions_list = questions_data['questions']
                params = {
                    "parent_id": f"{file_path}-{i}",
                    "questions": []
                }

            for iq, q in enumerate(cleaned_questions_list):
                if isinstance(q, dict) and 'question' in q:
                    try:
                        question_text = q['question']
                        embedding = embeddings.embed_query(question_text)
                        params["questions"].append({"text": question_text, "id": f"{file_path}-{i}-{iq}", "embedding": embedding})
                    except Exception as e:
                        print(f"Error embedding question {q}: {e}")
                else:
                    print(f"Skipping invalid question entry: {q}")

                sleep(5)
                graph.query(
                    """
                    MERGE (p:Parent {id: $parent_id})
                    WITH p
                    UNWIND $questions AS question
                    MERGE (q:Question {id: question.id})
                    ON CREATE SET q.text = question.text
                    MERGE (q)<-[:HAS_QUESTION]-(p)
                    WITH q, question
                    CALL db.create.setVectorProperty(q, 'embedding', question.embedding)
                    YIELD node
                    RETURN count(*)
                    """,
                    params,
                )
                # Create vector index
                try:
                    graph.query(
                        "CALL db.index.vector.createNodeIndex('hypothetical_questions', "
                        "'Question', 'embedding', $dimension, 'cosine')",
                        {"dimension": embedding_dimension},
                    )
                except ClientError:  # already exists
                    pass
                
        for i, parent in enumerate(parent_documents):
            summary = summary_chain.invoke({"question": parent.page_content}).content
            params = {
                "parent_id": f"{file_path}-{i}",
                "summary": summary,
                "embedding": embeddings.embed_query(summary),
            }
            graph.query(
                """
                MERGE (p:Parent {id: $parent_id})
                MERGE (p)-[:HAS_SUMMARY]->(s:Summary)
                ON CREATE SET s.text = $summary
                WITH s
                CALL db.create.setVectorProperty(s, 'embedding', $embedding)
                YIELD node
                RETURN count(*)
                """,
                params,
            )
            # Create vector index
            try:
                graph.query(
                    "CALL db.index.vector.createNodeIndex('summary', "
                    "'Summary', 'embedding', $dimension, 'cosine')",
                    {"dimension": embedding_dimension},
                )
            except ClientError:  # already exists
                pass
            
    end = time.perf_counter()
    print(f"Pipeline completed in {end-start} seconds")


## Test Data Ingestion ##

In [19]:
#Clear KG from previous session
graph.refresh_schema()
graph.query("MATCH (n) DETACH DELETE n")
graph.query("DROP INDEX hypothetical_questions IF EXISTS")
graph.query("DROP INDEX parent_document IF EXISTS")
graph.query("DROP INDEX summary IF EXISTS")
graph.query("DROP INDEX typical_rag IF EXISTS")

[]

In [20]:
folders = ["aapl-20220924","aapl-20230930","goog-20231231","msft-20220630","msft-20230630","nvda-20220130","nvda-20230129","nvda-20240128"]
ingestion_pipeline(folders)

Running pipeline for 4 files in aapl-20220924 folder
Extracting entities and relationships for: ./sec_10K_data/aapl-20220924\1.txt
Extracting entities and relationships for: ./sec_10K_data/aapl-20220924\1A.txt
Extracting entities and relationships for: ./sec_10K_data/aapl-20220924\7.txt
Extracting entities and relationships for: ./sec_10K_data/aapl-20220924\7A.txt
Pipeline completed in 318.85228709999865 seconds
Running pipeline for 4 files in aapl-20230930 folder
Extracting entities and relationships for: ./sec_10K_data/aapl-20230930\1.txt
Extracting entities and relationships for: ./sec_10K_data/aapl-20230930\1A.txt
Extracting entities and relationships for: ./sec_10K_data/aapl-20230930\7.txt
Extracting entities and relationships for: ./sec_10K_data/aapl-20230930\7A.txt
Pipeline completed in 326.5033165999994 seconds
Running pipeline for 4 files in goog-20231231 folder
Extracting entities and relationships for: ./sec_10K_data/goog-20231231\1.txt
Extracting entities and relationships 

## Test RAG ##

In [27]:
from langchain_community.vectorstores import Neo4jVector


# Typical RAG retriever

typical_rag = Neo4jVector.from_existing_index(
    embeddings, index_name="typical_rag"
)

# Parent retriever

parent_query = """
MATCH (node)<-[:HAS_CHILD]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata LIMIT 1
"""

parent_vectorstore = Neo4jVector.from_existing_index(
    embeddings,
    index_name="parent_document",
    retrieval_query=parent_query,
)

# Hypothetic questions retriever

hypothetic_question_query = """
MATCH (node)<-[:HAS_QUESTION]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata
"""

hypothetic_question_vectorstore = Neo4jVector.from_existing_index(
    embeddings,
    index_name="hypothetical_questions",
    retrieval_query=hypothetic_question_query,
)
# Summary retriever

summary_query = """
MATCH (node)<-[:HAS_SUMMARY]-(parent)
WITH parent, max(score) AS score // deduplicate parents
RETURN parent.text AS text, score, {} AS metadata
"""

summary_vectorstore = Neo4jVector.from_existing_index(
    embeddings,
    index_name="summary",
    retrieval_query=summary_query,
)

In [29]:
response = typical_rag.similarity_search(
    "What are some risk factors that can affect Apple's stock price?"
)
print(response[0].page_content)

 Item 1A. Risk Factors 

The Company&#8217;s business, reputation, results of operations, financial condition and stock price can be affected by a number of factors, whether currently known or unknown, including those described below. When any one or more of these risks materialize from time to time, the Company&#8217;s business, reputation, results of operations, financial condition and stock price can be materially and adversely affected. 

Because of the following factors, as well as other factors affecting the Company&#8217;s results of operations and financial condition, past financial performance should not be considered to be a reliable indicator of future performance, and investors should not use historical trends to anticipate results or trends in future periods. This discussion of risk factors contains forward-looking statements. 

This section should be read in conjunction with Part II, Item 7, &#8220;Management&#8217;s Discussion and Analysis of Financial Condition and Resu

In [31]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI

vector_typrag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=typical_rag.as_retriever()
)

vector_parent = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=parent_vectorstore.as_retriever()
)

vector_hypquestion = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=hypothetic_question_vectorstore.as_retriever()
)

vector_summary = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=summary_vectorstore.as_retriever()
)

vector_typrag.invoke(
     "What are some risk factors that can affect Apple's stock price?"
)

{'query': "What are some risk factors that can affect Apple's stock price?",
 'result': "There are several risk factors that can affect Apple's stock price, including:\n\n1. Macroeconomic and industry risks, such as adverse economic conditions, slow growth or recession, high unemployment, inflation, tighter credit, higher interest rates, and currency fluctuations.\n\n2. Political events, trade and other international disputes, war, terrorism, natural disasters, public health issues, industrial accidents, and other business interruptions.\n\n3. Global markets for the Company's products and services are highly competitive and subject to rapid technological change, and the Company may be unable to compete effectively in these markets.\n\n4. The Company's new products often utilize custom components available from only one source, and the continued availability of these components at acceptable prices, or at all, can be affected for any number of reasons.\n\n5. Losses or unauthorized acces

In [32]:
vector_hypquestion.invoke(
     "What are some risk factors that can affect Apple's stock price?"
)

{'query': "What are some risk factors that can affect Apple's stock price?",
 'result': "There are several risk factors that can affect Apple's stock price, as outlined in the company's 2023 Form 10-K. These include macroeconomic and industry risks, political events, trade and other international disputes, war, terrorism, natural disasters, public health issues, industrial accidents, and other business interruptions. Additionally, the company faces significant competition in global markets for its products and services, and its ability to compete successfully depends heavily on ensuring the continuing and timely introduction of innovative new products, services, and technologies to the marketplace. Other risk factors include the need to manage frequent introductions and transitions of products and services, the dependence on carriers, wholesalers, retailers, and other resellers, and the exposure to credit risk and fluctuations in the values of its investment portfolio. Finally, the com

In [33]:
vector_parent.invoke(
     "What are some risk factors that can affect Apple's stock price?"
)

{'query': "What are some risk factors that can affect Apple's stock price?",
 'result': "There are several risk factors that can affect Apple's stock price, including adverse macroeconomic conditions, political events, trade and other international disputes, war, terrorism, natural disasters, public health issues, industrial accidents, and other business interruptions. Additionally, global markets for Apple's products and services are highly competitive and subject to rapid technological change, and the company may be unable to compete effectively in these markets. The company's ability to continually improve its products and services to maintain their functional and design advantages is also a risk factor. Finally, to remain competitive and stimulate customer demand, the company must successfully manage frequent introductions and transitions of products and services."}

##RAG Agent Tool (combination of retrievers)

In [36]:
from langchain.chains import GraphCypherQAChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType


cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = AzureChatOpenAI(azure_deployment='Chat_gpt_4',api_version="2023-05-15", temperature=0),
    qa_llm = AzureChatOpenAI(azure_deployment='chat_gtp_35',api_version="2023-05-15", temperature=0), graph=graph, verbose=True,
)

tools = [
    Tool(
        name="Tasks",
        func=vector_typrag.run,
        description="""Useful to answer most of the questions.
        Not useful for questions that involve aggregation.
        Use full question as input.
        """,
        
    ),
    Tool(
        name="Tasks",
        func=vector_hypquestion.run,
        description="""Useful to answer questions on dates and relationship between different companies.
        Not useful for questions that involve aggregation.
        Use full question as input.
        """,        
        
    ),
    # Tool(
    #     name="Graph",
    #     func=cypher_chain.run,
    #     description=""" Only useful for AGGREGATION questions.
    #     Use full question as input.
    #     """,
    # ),
]

mrkl = initialize_agent(
    tools, 
    AzureChatOpenAI(azure_deployment='Chat_gpt_4',api_version="2023-05-15", temperature=0),
    agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)


C:\Users\lriospie\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
C:\Users\lriospie\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [37]:
graph.refresh_schema()
response = mrkl.invoke("What are some risk factors that can affect Apple's stock price?")
print(response)



> Entering new AgentExecutor chain...
Several risk factors can affect Apple Inc.'s stock price, including but not limited to:

1. **Market Competition**: Apple operates in highly competitive markets, including smartphones, personal computers, tablets, wearables, and digital services. Intense competition from other tech giants and emerging players can impact Apple's market share and profitability.

2. **Supply Chain Disruptions**: Apple relies on a global supply chain for the manufacturing of its products. Disruptions due to geopolitical tensions, natural disasters, pandemics, or labor disputes can affect product availability and increase costs.

3. **Regulatory Risks**: Changes in regulations, especially in key markets like the United States, Europe, and China, can impact Apple's operations. This includes antitrust investigations, privacy regulations, and taxes on digital services.

4. **Economic Conditions**: Economic downturns or recessions can lead to reduced consumer spending, af

In [38]:
graph.refresh_schema()
response = mrkl.invoke("What kind of business Apple handles?")
print(response)



> Entering new AgentExecutor chain...

Invoking: `Tasks` with `What kind of business Apple handles?`


Apple designs, manufactures, and markets smartphones, personal computers, tablets, wearables and accessories, and sells a variety of related services. The company also offers digital content through subscription-based services, payment services, and advertising services. Apple manages its business primarily on a geographic basis and sells its products and resells third-party products in most of its major markets directly to customers through its retail and online stores and its direct sales force.Apple is involved in designing, manufacturing, and marketing a wide range of products and services. These include:

- **Smartphones**: Apple's iPhone is one of the most popular smartphone brands globally.
- **Personal Computers**: The company produces the Mac line of computers.
- **Tablets**: Apple's iPad is a leading product in the tablet market.
- **Wearables and Accessories**: This inclu

In [3]:
# pip freeze >requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
# import sys
# print("Python version: " + sys.version)

Python version: 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
